**주제**
  - 기후변화에 따른 온열질환자수 증가와 취약계층과의 상관관계 분석

### **준비**

- 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 각종 라이브러리 import

In [280]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

- 데이터 파일 경로 설정

In [281]:
# 프로젝트 파일 구성 /content/drive/MyDrive/Mid_project/
# DATA_PATH = "/content/drive/MyDrive/Mid_project/data/"

DATA_PATH = "/content/drive/MyDrive/Mid_project/data/"

## **데이터 수집 및 전처리**

### **1. 기온 관련 데이터 수집 및 전처리** (김민주)



In [345]:
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/원본_데이터/기온/"

- 원본 데이터 전처리 및 병합

In [346]:
# 지역 17개 명칭
file_names = ['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시',
              '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시',
              '인천광역시', '전라남도', '전라북도', '제주', '충청남도', '충청북도']

all_data = []
# 연도별로 데이터를 처리하여 병합
for year in range(2019, 2024+1):  # 2019년부터 2024년까지 반복
    folder_path = f"{DATA_PATH}{year}_기온/"  # 연도별 폴더 경로
    df_list = []

    for file_name in file_names:
        file_path = f"{folder_path}{year}_{file_name}.csv"

        if file_name in ['강원도', '경기도']:
            df_list.append(pd.read_csv(file_path))  # 인코딩 처리 불필요
        else:
            df_list.append(pd.read_csv(file_path, encoding='cp949'))  # 다른 파일은 cp949 인코딩 처리

    # 연도별 데이터 병합
    year_df = pd.concat(df_list, ignore_index=True)

    # 날짜 컬럼 전처리
    year_df['날짜'] = year_df['날짜'].str.strip().str.replace('\t', '')

    # 행정코드(지점) 컬럼 전처리
    region_code_map = {108: "서울특별시",
                       156: "광주광역시",
                       143: "대구광역시",
                       133: "대전광역시",
                       159: "부산광역시",
                       239: "세종특별자치시",
                       152: "울산광역시",
                       112: "인천광역시",
                       "경남": "경상남도",
                       "경북": "경상북도",
                       "전남": "전라남도",
                       "전북": "전라북도",
                       "충남": "충청남도",
                       "충북": "충청북도",
                       "제주": "제주특별자치도"}
    year_df["지점"] = year_df["지점"].replace(region_code_map)

    # 전처리 완료된 연도별 데이터를 리스트에 추가
    all_data.append(year_df)

# 전체 데이터를 하나의 데이터프레임으로 병합
기온데이터 = pd.concat(all_data, ignore_index=True)

# 날짜 데이터 분리
기온데이터['년도'] = pd.to_datetime(기온데이터['날짜']).dt.year
기온데이터['월'] = pd.to_datetime(기온데이터['날짜']).dt.month
기온데이터['일'] = pd.to_datetime(기온데이터['날짜']).dt.day


# 폭염여부 feature 추가
기온데이터['폭염여부'] = (기온데이터['최고기온(℃)'] >= 33).astype(int)
# 일교차 feature 추가
기온데이터['일교차'] = 기온데이터['최고기온(℃)']-기온데이터['최저기온(℃)']

# ℃ 제거
기온데이터.columns = 기온데이터.columns.str.replace(r'\(℃\)', '', regex=True)

기온데이터['날짜'] = pd.to_datetime(기온데이터['날짜'])

# 5월 20일 ~ 9월 30일의 데이터만 필터링(행안부 선정 폭염 집중 대응기간)
기온데이터 = 기온데이터[(기온데이터['날짜'].dt.strftime('%m-%d') >= '05-20') & (기온데이터['날짜'].dt.strftime('%m-%d') <= '09-30')]

# 컬럼 정
cols = ['날짜', '년도', '월', '일', '지점', '평균기온', '최저기온', '최고기온', '폭염여부', '일교차']
기온데이터 = 기온데이터[cols]

# 날짜별로 정렬
기온데이터.sort_values(by='날짜', inplace=True)
기온데이터.reset_index(drop=True, inplace=True)

- 결측치 확인 중 5개의 결측치 발견"
  - 해당 값을 같은 날짜의 충청북도와 충청남도의 평균기온의 평균으로 대체
  - 해당 값을 동일 지역의 같은 날짜의 값으로 변경

In [347]:
기온데이터[기온데이터.isnull().any(axis=1)]

,날짜,년도,월,일,지점,평균기온,최저기온,최고기온,폭염여부,일교차
190,2019-05-31,2019.0,5.0,31.0,세종특별자치시,NaN,13.7,24.1,0,10.4
8188,2022-08-08,2022.0,8.0,8.0,서울특별시,26.8,NaN,28.4,0,NaN
9070,2022-09-29,2022.0,9.0,29.0,세종특별자치시,NaN,15.1,NaN,0,NaN
9086,2022-09-30,2022.0,9.0,30.0,세종특별자치시,NaN,12.9,27.8,0,14.9
9179,2023-05-24,2023.0,5.0,24.0,부산광역시,18.5,NaN,22.9,0,NaN


In [348]:
# 2019-05-31 세종특별자치시의 평균기온 결측치 채우기 (2019-05-31 충청남도와 충청북도의 값을 가져옴)
평균기온_0531_남 = 기온데이터[(기온데이터['지점'] == '충청남도') & (기온데이터['날짜'] == '2019-05-31')]['평균기온'].values[0]
평균기온_0531_북 = 기온데이터[(기온데이터['지점'] == '충청북도') & (기온데이터['날짜'] == '2019-05-31')]['평균기온'].values[0]
평균_0531 = (평균기온_0531_남 + 평균기온_0531_북) / 2

#2022-09-29 세종특별자치시의 평균기온 결측치 채우기 (2022-09-29 충청남도와 충청북도의 값을 가져옴)
평균기온_0929_남 = 기온데이터[(기온데이터['지점'] == '충청남도') & (기온데이터['날짜'] == '2022-09-29')]['평균기온'].values[0]
평균기온_0929_북 = 기온데이터[(기온데이터['지점'] == '충청북도') & (기온데이터['날짜'] == '2022-09-29')]['평균기온'].values[0]
평균_0929 = (평균기온_0929_남 + 평균기온_0929_북) / 2

#2022-09-30 세종특별자치시의 평균기온 결측치 채우기 (2022-09-30 충청남도와 충청북도의 값을 가져옴)
평균기온_0930_남 = 기온데이터[(기온데이터['지점'] == '충청남도') & (기온데이터['날짜'] == '2022-09-30')]['평균기온'].values[0]
평균기온_0930_북 = 기온데이터[(기온데이터['지점'] == '충청북도') & (기온데이터['날짜'] == '2022-09-30')]['평균기온'].values[0]
평균_0930 = (평균기온_0930_남 + 평균기온_0930_북) / 2


#평균기온 NaN 값채우기
nan_indices = 기온데이터[기온데이터['평균기온'].isna()].index

if len(nan_indices) >= 3:
  기온데이터.loc[nan_indices[0], '평균기온'] = 평균_0531
  기온데이터.loc[nan_indices[1], '평균기온'] = 평균_0929
  기온데이터.loc[nan_indices[2], '평균기온'] = 평균_0930


  #최저기온 NaN 값 채우기
nan_indices = 기온데이터[기온데이터['최저기온'].isna()].index

if len(nan_indices) >= 2 :
  기온데이터.loc[nan_indices[0], '최저기온'] = 기온데이터[(기온데이터['지점'] == '경기도') & (기온데이터['날짜'] == '2019-05-31')]['최저기온'].values[0]
  기온데이터.loc[nan_indices[1], '최저기온'] = 기온데이터[(기온데이터['지점'] == '경상남도') & (기온데이터['날짜'] == '2023-05-24')]['최저기온'].values[0]


# 최고기온 NaN 값 채우기
최고기온_0929_남 = 기온데이터[(기온데이터['지점'] == '충청남도') & (기온데이터['날짜'] == '2022-09-29')]['최고기온'].values[0]
최고기온_0929_북 = 기온데이터[(기온데이터['지점'] == '충청북도') & (기온데이터['날짜'] == '2022-09-29')]['최고기온'].values[0]

최고_0929 = (최고기온_0929_남 + 최고기온_0929_북) / 2

기온데이터['최고기온'] = 기온데이터['최고기온'].fillna(최고_0929)


# 일교차 NaN값 채우기
기온데이터['일교차'] = 기온데이터['최고기온']-기온데이터['최저기온']

### **2. 온열질환 관련 데이터 수집 및 전처리** (길선종)

In [349]:
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/원본_데이터/온열/"

- 연도별_나이별_장소별_온열환자별 환자수 데이터 전처리 및 병합

In [350]:
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/원본_데이터/온열/"

# 온열질환_나이별 데이터 전처리
온열질환_나이별 = pd.read_excel(f"{DATA_PATH}2011-2023_연도별_성별_연령별_신고현황.xlsx")
# 성별, 소계, 합계 등 필요없는 컬럼 및 행 drop
온열질환_나이별 = 온열질환_나이별.dropna()
온열질환_나이별 = 온열질환_나이별.drop("성별", axis=1)

# 온열질환_장소별 데이터 전처리
온열질환_장소별 = pd.read_excel(f"{DATA_PATH}연도별_장소별_온열환자발생.xlsx")
온열질환_장소별 = 온열질환_장소별.dropna()

온열질환_장소별.columns = 온열질환_장소별.iloc[0]  # 첫 번째 행을 컬럼명으로 사용
온열질환_장소별 = 온열질환_장소별.iloc[2:, :]  # 필요 없는 행 제거

# 온열질환 나이별, 장소별 데이터 merge 및 전처리
온열질환데이터 = pd.merge(온열질환_나이별, 온열질환_장소별, on="연도")
온열질환데이터 = 온열질환데이터.iloc[:, :]

# 컬럼 중복 제거 및 이름 수정
온열질환데이터.columns = ['연도', '온열질환자수합계', '0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69',
                               '70-79', '80+', '65+', '장소별합계', '소계1', '집', '건물', '실내작업장', '비닐하우스', '실내기타',
                               '소계2', '실외작업장', '운동장(공원)', '논/밭', '산/강/해변', '길가', '주거지주변', '실외기타']
# 중복된 '소계' 컬럼 제거
온열질환데이터 = 온열질환데이터.drop(['소계1', '소계2'], axis=1)

# 결측치 '-'을 0으로 대체하고 dtype을 추론하여 변환
온열질환데이터 = 온열질환데이터.replace("-", 0)

# inplace=True 대신, 결과를 원본에 다시 할당하는 방식
온열질환데이터['연도'] = 온열질환데이터['연도'].replace('년', '', regex=True)

# astype(int) 변환도 inplace 없이 적용
온열질환데이터['연도'] = 온열질환데이터['연도'].astype(int)
온열질환데이터 = 온열질환데이터[온열질환데이터['연도'] >= 2019]

# reset_index도 inplace 사용 대신 재할당
온열질환데이터 = 온열질환데이터.reset_index(drop=True)

<ipython-input-350-e80431ff61d0>:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  온열질환데이터 = 온열질환데이터.replace("-", 0)


- 기온데이터_연도 피벗테이블 생성

In [351]:
df = pd.pivot_table(
    기온데이터,
    index='년도',  # 행 기준은 '년도'
    values=['폭염여부', '평균기온', '일교차', '최고기온'],  # 집계할 컬럼들
    aggfunc={
        '폭염여부': 'sum',         # 폭염 여부의 합
        '평균기온': 'mean',        # 평균기온의 평균
        '일교차': 'mean',          # 일교차의 평균
        '최고기온': 'max'          # 최고기온의 최대값
    }
)
df.reset_index(inplace=True)
df.rename(columns={'년도': '연도'}, inplace=True)
df = df[df['연도'] < 2024]
df

,연도,일교차,최고기온,평균기온,폭염여부
0,2019.0,8.411606,36.9,23.557512,205
1,2020.0,7.765544,37.0,22.930601,129
2,2021.0,8.033894,37.2,23.502722,195
3,2022.0,8.189504,37.8,23.892120,182
4,2023.0,7.810777,37.7,24.131826,212


In [352]:
# 24년도 데이터는 없음...최근 5개년만 봐야할듯
온열질환데이터 = pd.merge(df, 온열질환데이터, on=['연도'], how='left')


온열질환데이터['일교차'] = 온열질환데이터['일교차'].round(1)
온열질환데이터['평균기온'] = 온열질환데이터['평균기온'].round(1)
온열질환데이터['연도'] = 온열질환데이터['연도'].astype(int)
온열질환데이터

,연도,일교차,최고기온,평균기온,폭염여부,온열질환자수합계,0-9,10-19,20-29,30-39,...,실내작업장,비닐하우스,실내기타,실외작업장,운동장(공원),논/밭,산/강/해변,길가,주거지주변,실외기타
0,2019,8.4,36.9,23.6,205,1841,8,81,192,221,...,125,27,44,596,126,269,86,198,75,126
1,2020,7.8,37.0,22.9,129,1078,0,24,103,100,...,62,21,9,378,47,212,46,132,52,40
2,2021,8.0,37.2,23.5,195,1376,1,39,141,120,...,98,18,31,555,53,159,52,137,50,90
3,2022,8.2,37.8,23.9,182,1564,3,42,168,176,...,127,22,28,581,95,230,58,161,57,103
4,2023,7.8,37.7,24.1,212,2818,16,95,291,323,...,197,48,87,913,171,395,104,286,105,269


In [353]:
온열질환데이터.to_csv("/content/drive/MyDrive/Mid_project/data/가공_데이터/온열질환데이터.csv",index=False, encoding='utf-8')

- 온열질환_도시별 환자수 데이터 전처리 및 병합

In [354]:
온열환자_도시별 = pd.read_csv(f"{DATA_PATH}도시별_온열질환자_현황_2021-2024.csv")

# 증상발생일에 합쳐져 있는 합계 drop(전체 합계는 필요가 없음)
온열환자_도시별 = 온열환자_도시별[온열환자_도시별['증상발생일']!='합계']

#  증상 발생일 columns 타입 변경 밑 columns 명 변경(통일을위해
온열환자_도시별['증상발생일'] = pd.to_datetime(온열환자_도시별['증상발생일'])
온열환자_도시별.rename(columns={'증상발생일':'날짜'}, inplace=True)

# 환자수, 추정사망자수 합계 Drop(불필요한 data)
온열환자_도시별.drop(columns=['추정사망자수', '환자수'], inplace=True)

# 온열환자_도시별

- 온열질환데이터(2021년,2022년, 2023년, 2024년의  5월 20~9월 30일 폭염 집중 대응기간의 온열질환 환자수 데이터

In [355]:
# 온열환자_도시별 데이터 변환
# 기온데이터와 merge 하여 기온과 온열질환자수의 상관관계 분석
지역구_목록 = ['강원도', '경상북도', '광주광역시', '대구광역시', '경기도', '대전광역시', '부산광역시', '서울특별시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도', '경상남도','세종특별자치시']

온열환자_도시별= 온열환자_도시별.melt(id_vars=['날짜'], value_vars=지역구_목록, var_name='지점', value_name='온열질환_환자수')
온열환자_도시별

,날짜,지점,온열질환_환자수
0,2021-05-20,강원도,0
1,2021-05-21,강원도,0
2,2021-05-22,강원도,0
3,2021-05-23,강원도,0
4,2021-05-24,강원도,0
...,...,...,...
9107,2024-09-26,세종특별자치시,0
9108,2024-09-27,세종특별자치시,0
9109,2024-09-28,세종특별자치시,0
9110,2024-09-29,세종특별자치시,0


In [356]:
기온데이터 = pd.merge(기온데이터, 온열환자_도시별, on=['날짜', '지점'], how='right')
기온데이터.to_csv("/content/drive/MyDrive/Mid_project/data/가공_데이터/기온데이터.csv", index=False, encoding='utf-8')

### **3. 기타 관련 데이터 수집 및 전처리** (이채희)

In [357]:
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/원본_데이터/기타/"

- 연도별_지역별_3대만성질환(고혈압, 당뇨, 이상지질혈증) 데이터 전처리

In [358]:
# 만성질환 데이터 로드
disease_population = pd.read_csv(f"{DATA_PATH}질병관리청_3대만성질환(고혈압,당뇨,이상지질혈증)_진단경험률.csv", encoding='utf-8')

# 다중컬럼 및 다중인덱스 데이터프레임으로 변형 및 컬럼 재설정
disease_population = disease_population.set_index(['시도', '시군구'])
disease_population.columns = disease_population.columns.str.replace("Unnamed: ", "")
tmp = [x for x in list(disease_population.columns) if int(x) > 2000]
disease_population.index.names = ['시도', '시군구'] # index 설정
disease_population.columns = [disease_population.columns.map(lambda x : x if int(x) > 2000 else tmp[(int(x)-2)//12]), disease_population.iloc[0]]
disease_population = disease_population.iloc[1:]
disease_population.columns.names = ['연도','구분']


# 결측치 확인
disease_population = disease_population.astype('float')
disease_population = disease_population.replace(0, np.NaN)

# 시도별 데이터 통합 -> 평균화
new_data = disease_population.groupby(['시도']).mean()


# 시도별 고혈압 데이터
high_blood_rate = new_data.xs(key='고혈압 진단 경험률(30세 이상)_조율',level=1, axis=1)
high_blood_rate = high_blood_rate.fillna(high_blood_rate.mean())
high_blood_rate.reset_index(inplace=True)
high_blood_rate.rename(columns={'시도': '행정구역'}, inplace=True)
고혈압_진단경험률 = high_blood_rate

# 시도별 당뇨병 데이터
diabetes_rate = new_data.xs(key='당뇨병 진단 경험률(30세 이상)_조율',level=1, axis=1)
diabetes_rate = diabetes_rate.fillna(diabetes_rate.mean())
diabetes_rate.reset_index(inplace=True)
diabetes_rate.rename(columns={'시도': '행정구역'}, inplace=True)
당뇨_진단경험률 = diabetes_rate

- 연도별_지역별_농가인구_및_어가인구 데이터 전처리

In [359]:
# 농가인구 데이터 로드
farming_population = pd.read_csv(f"{DATA_PATH}행정구역_시군구_별_농가__농가인구_20241011141838.csv", encoding='cp949')
farming_population = farming_population.set_index('행정구역별(1)')
farming_population.index.names = ['행정구역별'] # index 설정
farming_population.columns = [farming_population.columns.map(lambda x : x[:4]), farming_population.iloc[0]]
farming_population = farming_population.iloc[1:]
farming_population.columns.names = ['연도','구분']

# 어가인구 데이터 로드
fishing_population = pd.read_csv(f"{DATA_PATH}시도별_어가__어가인구__어업종사가구원_20241011141818.csv", encoding='cp949')
fishing_population = fishing_population.set_index('시도별')
fishing_population.index.names = ['시도별'] # index 설정
fishing_population.columns = [fishing_population.columns.map(lambda x : x[:4]), fishing_population.iloc[0]]
fishing_population = fishing_population.iloc[1:]
fishing_population.columns.names = ['연도','구분']

# 산업별 총 종사자 수 컬럼 추출
new_farming_population = farming_population.xs(key='농가인구 (명)',level=1, axis=1)
new_fishing_population = fishing_population.xs(key='어가인구 (명)',level=1, axis=1)


# outer join 병합 및 개별 값 전처리
outer_data = new_farming_population.join(new_fishing_population, how = 'outer', lsuffix="_농가인구",rsuffix='_어가인구')
outer_data = outer_data.fillna(0)
outer_data = outer_data.astype(int)

# 농어가 인구 합산
cols = list(new_farming_population.columns)
for e in cols:
    outer_data[e] = outer_data[f'{e}_농가인구'] + outer_data[f'{e}_어가인구']

# 농어가 합계 인구
outer_data_sum = outer_data.loc[:,['2017','2018','2019','2020','2021','2022','2023']]
농어가합계인구 = outer_data_sum.reset_index()
농어가합계인구.rename(columns={'index': '행정구역'}, inplace=True)

### **4. 인구 관련 데이터 수집 및 전처리** (천대원)

In [360]:
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/원본_데이터/인구/"

df = pd.read_csv(f"{DATA_PATH}시도별_연령별_인구현황(연간).csv", encoding='cp949')

# 인구수 쉼표 제거 및 정수형으로 변환
df_numeric = df.drop(columns=['행정구역']).apply(lambda x: x.str.replace(',', '').astype('int64'))
df_final = pd.concat([df['행정구역'], df_numeric], axis=1)

# 중복되는 지역 제거
df = df_final.drop(index=11)

# 행정구역의 지역코드 제거
df['행정구역'] = df['행정구역'].str.replace(r'\s*\(.*?\)', '', regex=True)

# 불필요한 컬럼 제거
col_lst = ["2019년_계_연령구간인구수", "2020년_계_연령구간인구수", "2021년_계_연령구간인구수",
           "2022년_계_연령구간인구수", "2023년_계_연령구간인구수"]
df = df.drop(columns=col_lst)

# 불필요한 문자열 제거
lst = df.columns[1:]
tmp = []
for i in lst:
    tmp.append(i.replace("계_", ""))
tmp.insert(0, "행정구역")
df.columns = tmp

# 10세 미만, 60세 이상 등 폭염 취약연령 비율 추가
year = ["2019", "2020", "2021", "2022", "2023"]

for y in year:
    df[f"{y}년_취약연령"] = df[f"{y}년_0~9세"]+ df[f"{y}년_60~69세"] + df[f"{y}년_70~79세"] + df[f"{y}년_80~89세"] + df[f"{y}년_90~99세"] + df[f"{y}년_100세 이상"]
    df[f"{y}년_취약연령 비율"] = round((df[f"{y}년_취약연령"] / df[f"{y}년_총인구수"]), 2)

# 취약연령 수 제거
lst = ["2019년_취약연령",
       "2020년_취약연령",
       "2021년_취약연령",
       "2022년_취약연령",
       "2023년_취약연령"]
df = df.drop(columns=lst)

In [361]:
# 컬럼 정렬
cols = ['행정구역', '2019년_총인구수', '2019년_0~9세', '2019년_10~19세', '2019년_20~29세',
       '2019년_30~39세', '2019년_40~49세', '2019년_50~59세', '2019년_60~69세',
       '2019년_70~79세', '2019년_80~89세', '2019년_90~99세', '2019년_100세 이상',
        '2019년_취약연령 비율',
       '2020년_총인구수', '2020년_0~9세', '2020년_10~19세', '2020년_20~29세',
       '2020년_30~39세', '2020년_40~49세', '2020년_50~59세', '2020년_60~69세',
       '2020년_70~79세', '2020년_80~89세', '2020년_90~99세', '2020년_100세 이상',
        '2020년_취약연령 비율',
       '2021년_총인구수', '2021년_0~9세', '2021년_10~19세', '2021년_20~29세',
       '2021년_30~39세', '2021년_40~49세', '2021년_50~59세', '2021년_60~69세',
       '2021년_70~79세', '2021년_80~89세', '2021년_90~99세', '2021년_100세 이상',
        '2021년_취약연령 비율',
       '2022년_총인구수', '2022년_0~9세', '2022년_10~19세', '2022년_20~29세',
       '2022년_30~39세', '2022년_40~49세', '2022년_50~59세', '2022년_60~69세',
       '2022년_70~79세', '2022년_80~89세', '2022년_90~99세', '2022년_100세 이상',
        '2022년_취약연령 비율',
       '2023년_총인구수', '2023년_0~9세', '2023년_10~19세', '2023년_20~29세',
       '2023년_30~39세', '2023년_40~49세', '2023년_50~59세', '2023년_60~69세',
       '2023년_70~79세', '2023년_80~89세', '2023년_90~99세', '2023년_100세 이상',
       '2023년_취약연령 비율']

df = df[cols]

# 연도별 분리 (2021년~2023년만 사용)
df_2021 = df[[col for col in df.columns if '2021' in col or col == '행정구역']]

df_2022 = df[[col for col in df.columns if '2022' in col or col == '행정구역']]

df_2023 = df[[col for col in df.columns if '2023' in col or col == '행정구역']]

# 각 연도별 컬럼명 재정의
lst = ['행정구역', '총인구수', '0~9세', '10~19세', '20~29세',
       '30~39세', '40~49세', '50~59세', '60~69세',
       '70~79세', '80~89세', '90~99세', '100세 이상',
       '취약연령 비율']
df_2021.columns = lst
df_2022.columns = lst
df_2023.columns = lst

# 행정구역 기준 정렬
lst = [df_2021, df_2022, df_2023]
for df in lst:
    df.sort_values("행정구역", ascending=True, inplace=True)

In [362]:
# 농어가 합계 인구에 대한 비율 추가하기
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/가공_데이터/"

df = 농어가합계인구
농어가합계인구.rename(columns={"Unnamed: 0": "행정구역"}, inplace=True)

# 해당 연도별 농어가 합계 인구 merge
years = [2021, 2022, 2023]
dfs = {2021: df_2021, 2022: df_2022, 2023: df_2023}

for year in years:
    # 데이터 merge
    dfs[year] = pd.merge(dfs[year], df[["행정구역", str(year)]], on="행정구역", how="left")
    # 농어가 인구 비율 계산
    dfs[year][f"농어가 인구 비율"] = round(dfs[year][str(year)] / dfs[year]["총인구수"], 2)
    dfs[year].rename(columns={str(year): "농어가 인구"}, inplace=True)

df_2021, df_2022, df_2023 = dfs[2021], dfs[2022], dfs[2023]

In [363]:
# 5대 질병(당뇨병, 고혈압) 추가
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/가공_데이터/"

high_blood_pressure = 고혈압_진단경험률
diabetes = 당뇨_진단경험률

high_blood_pressure.drop(columns=['2020', '2019', '2018', '2017'], inplace=True)
diabetes.drop(columns=['2020', '2019', '2018', '2017'], inplace=True)

lst = ['2023', '2022', '2021']

for i in lst:
    high_blood_pressure[i] = round(high_blood_pressure[i] / 100, 2)
    diabetes[i] = round(diabetes[i] / 100, 2)

high_blood_pressure.rename(columns={"시도":"행정구역"}, inplace=True)
diabetes.rename(columns={"시도":"행정구역"}, inplace=True)

years = [2021, 2022, 2023]
dfs = {2021: df_2021, 2022: df_2022, 2023: df_2023}

for year in years:
    # 고혈압 경험 비율
    dfs[year] = pd.merge(dfs[year], high_blood_pressure[["행정구역", str(year)]], on="행정구역", how='left')
    dfs[year] = dfs[year].rename(columns={str(year): "고혈압 경험 비율"})
    # 당뇨 경험 비율
    dfs[year] = pd.merge(dfs[year], diabetes[["행정구역", str(year)]], on="행정구역", how='left')
    dfs[year] = dfs[year].rename(columns={str(year): "당뇨 경험 비율"})

df_2021, df_2022, df_2023 = dfs[2021], dfs[2022], dfs[2023]

In [364]:
# 노후주택 비율
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/원본_데이터/인구/"
df = pd.read_csv(f"{DATA_PATH}주택의_종류__연면적_및_건축연도별_주택__시군구_20241014130834.csv")

df.drop(index=0, inplace=True)

df_int = df[['2021', '2021.1', '2022',
       '2022.1', '2023', '2023.1']].astype(int)

df.drop(columns=['2021', '2021.1', '2022',
       '2022.1', '2023', '2023.1'], inplace=True)

df = pd.concat([df, df_int], axis=1)

# 연도별 지역별 주택노후 비율
for year in years:
    df[f"{year}_노후주택 비율"] = round(df[f"{year}.1"] / df[f"{year}"], 2)

df.rename(columns={"행정구역별(시군구)(1)":"행정구역"}, inplace=True)
df.sort_values("행정구역", inplace=True)
df['행정구역'] = df['행정구역'].replace('강원특별자치도', '강원도')

# 연도별 지역별 주택노후 비율 merge
for year in years:
    dfs[year] = pd.merge(dfs[year], df[["행정구역", f"{year}_노후주택 비율"]], on="행정구역", how='left')
    dfs[year].rename(columns={f"{year}_노후주택 비율": "노후주택 비율"}, inplace=True)

df_2021, df_2022, df_2023 = dfs[2021], dfs[2022], dfs[2023]

In [365]:
df_2021['연도'] = 2021
df_2022['연도'] = 2022
df_2023['연도'] = 2023

# 세 데이터프레임을 하나로 합치기
merged_df = pd.concat([df_2021, df_2022, df_2023], ignore_index=True)

# 합계열 drop
merged_df = merged_df[merged_df["행정구역"] != "전국"]

# columns 정리
cols = ['연도'] + [col for col in merged_df.columns if col != '연도']
merged_df = merged_df[cols]
merged_df = merged_df.reset_index(drop=True)
인구데이터 = merged_df

In [369]:
인구데이터.to_csv("/content/drive/MyDrive/Mid_project/data/가공_데이터/인구데이터.csv", index=False, encoding='utf-8')

## **데이터 불러오기 및 시각화**

In [370]:
DATA_PATH = "/content/drive/MyDrive/Mid_project/data/가공_데이터/"
기온데이터 = pd.read_csv(f"{DATA_PATH}기온데이터.csv")
온열질환데이터 = pd.read_csv(f"{DATA_PATH}온열질환데이터.csv")
인구데이터 = pd.read_csv(f"{DATA_PATH}인구데이터.csv")

In [277]:
기온데이터

,날짜,년도,월,일,지점,평균기온,최저기온,최고기온,폭염여부,일교차,온열질환_환자수
0,2021-05-20,2021.0,5.0,20.0,강원도,16.30,13.15,21.20,0,8.05,0
1,2021-05-21,2021.0,5.0,21.0,강원도,14.10,12.10,16.65,0,4.55,0
2,2021-05-22,2021.0,5.0,22.0,강원도,19.20,12.10,26.45,0,14.35,0
3,2021-05-23,2021.0,5.0,23.0,강원도,20.85,14.30,27.25,0,12.95,0
4,2021-05-24,2021.0,5.0,24.0,강원도,19.10,14.65,24.10,0,9.45,0
...,...,...,...,...,...,...,...,...,...,...,...
9107,2024-09-26,2024.0,9.0,26.0,세종특별자치시,23.30,20.00,28.50,0,8.50,0
9108,2024-09-27,2024.0,9.0,27.0,세종특별자치시,22.30,18.10,28.10,0,10.00,0
9109,2024-09-28,2024.0,9.0,28.0,세종특별자치시,22.90,18.30,27.90,0,9.60,0
9110,2024-09-29,2024.0,9.0,29.0,세종특별자치시,22.70,18.90,27.30,0,8.40,0


In [278]:
온열질환데이터

,연도,온열질환자수합계,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,실내작업장,비닐하우스,실내기타,실외작업장,운동장(공원),논/밭,산/강/해변,길가,주거지주변,실외기타
0,2019,1841,8,81,192,221,306,385,292,191,...,125,27,44,596,126,269,86,198,75,126
1,2020,1078,0,24,103,100,169,244,219,113,...,62,21,9,378,47,212,46,132,52,40
2,2021,1376,1,39,141,120,215,330,245,127,...,98,18,31,555,53,159,52,137,50,90
3,2022,1564,3,42,168,176,246,344,281,163,...,127,22,28,581,95,230,58,161,57,103
4,2023,2818,16,95,291,323,385,601,514,325,...,197,48,87,913,171,395,104,286,105,269


In [279]:
인구데이터

,연도,행정구역,총인구수,0~9세,10~19세,20~29세,30~39세,40~49세,50~59세,60~69세,70~79세,80~89세,90~99세,100세 이상,취약연령 비율,농어가 인구,농어가 인구 비율,고혈압 경험 비율,당뇨 경험 비율,노후주택 비율
0,2021,강원도,1538492,100971,133885,175424,162817,219422,264586,260158,132627,76586,11658,358,0.38,153596,0.10,0.35,0.15,0.11
1,2021,경기도,13565450,1099927,1321005,1798269,1900447,2296173,2274844,1669601,773573,374488,55363,1760,0.29,281612,0.02,0.26,0.11,0.02
2,2021,경상남도,3314183,250669,319461,362661,379886,527396,579779,494113,249720,131422,18682,394,0.35,261521,0.08,0.30,0.12,0.11
3,2021,경상북도,2626609,178515,221964,276021,279475,377180,454427,433047,244151,140963,20313,553,0.39,352785,0.13,0.31,0.14,0.17
4,2021,광주광역시,1441611,114065,152404,206472,181189,236167,235764,172124,94547,42636,6066,177,0.30,35403,0.02,0.23,0.10,0.05
5,2021,대구광역시,2385412,167823,218537,308921,284865,372555,419444,342291,177828,82781,10122,245,0.33,54006,0.02,0.25,0.12,0.06
6,2021,대전광역시,1452251,107548,142175,209397,189669,231018,240017,190514,92390,43199,6155,169,0.30,29025,0.02,0.26,0.11,0.05
7,2021,부산광역시,3350380,219611,266105,415549,405300,502391,554768,548878,298128,123791,15457,402,0.36,28318,0.01,0.26,0.12,0.09
8,2021,서울특별시,9509458,583683,756136,1422277,1439279,1480780,1507194,1285922,695800,295009,41735,1643,0.31,15767,0.00,0.25,0.10,0.05
9,2021,세종특별자치시,371895,47063,45794,38510,61719,72764,48876,33522,14775,7557,1288,27,0.28,15348,0.04,0.22,0.08,0.03
